# Storing training data on disk via Zarr

First we need some imports.

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import numpy as np
from scipy import stats
import pylab as plt
import torch
import torchist
import swyft

## Training data

Now we generate training data.  As simple example, we consider the model

$$
x = z + \epsilon
$$

where the parameter $z \sim \mathcal{N}(\mu = 0, \sigma = 1)$ is standard normal distributed, and $\epsilon \sim \mathcal{N}(\mu = 0, \sigma = 0.1)$ is a small noise contribution.  We are interested in the posterior of $z$ given a measurement of parameter $x$.

In [15]:
class Simulator(swyft.Simulator):
    def __init__(self):
        super().__init__()
        self.on_after_forward = swyft.to_numpy32

    def forward(self, trace):
        z = trace.sample('z', lambda: np.random.rand(1))
        x = trace.sample('x', lambda z: z + np.random.randn(1)*0.1, z)
        
sim = Simulator()
shapes, dtypes = sim.get_shapes_and_dtypes()

In [16]:
store = swyft.ZarrStore("./zarr_store2")
store.init(10000, 64, shapes, dtypes)

In [17]:
store.simulate(sim, batch_size = 1000)  # This function can be run in parallel in many threads

In [18]:
class Network(swyft.SwyftModule):
    def __init__(self):
        super().__init__()
        self.logratios = swyft.LogRatioEstimator_1dim(num_features = 1, num_params = 1, varnames = 'z')

    def forward(self, A, B):
        logratios = self.logratios(A['x'], B['z'])
        return logratios

## Trainer

Training is now done using the `SwyftTrainer` class, which extends `pytorch_lightning.Trainer` by methods like `infer` (see below).

In [23]:
trainer = swyft.SwyftTrainer(accelerator = 'gpu', gpus=1, max_epochs = 2, precision = 64)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


The `swyft.Samples` class provides convenience functions to generate data loaders for training and validation data.

In [24]:
dl_train = store.get_dataloader(batch_size = 64, idx_range = [0, 9500])
dl_valid = store.get_dataloader(batch_size = 64, idx_range = [9500, 10000])

In [25]:
network = Network()

In [26]:
trainer.fit(network, dl_train, dl_valid)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                   | Params
-----------------------------------------------------
0 | logratios | LogRatioEstimator_1dim | 17.4 K
-----------------------------------------------------
17.4 K    Trainable params
0         Non-trainable params
17.4 K    Total params
0.139     Total estimated model params size (MB)


Epoch 0: : 149it [00:01, 76.85it/s, loss=-0.551, v_num=9947768]
Validating: 0it [00:00, ?it/s]
Epoch 0: : 157it [00:01, 78.87it/s, loss=-0.551, v_num=9947768, val_loss=-.531]
Epoch 1: : 149it [00:02, 72.51it/s, loss=-0.563, v_num=9947768, val_loss=-.531]
Validating: 0it [00:00, ?it/s]
Epoch 1: : 157it [00:02, 74.54it/s, loss=-0.563, v_num=9947768, val_loss=-.535]
Epoch 1: : 158it [00:02, 74.14it/s, loss=-0.563, v_num=9947768, val_loss=-.535]
